## **PDF Data Loading**

Dependencias

[PyPDFLoader](https://python.langchain.com/docs/integrations/document_loaders/pypdfloader/)

In [1]:
%%capture
!pip install -qU langchain_community pypdf langchain-text-splitters

In [2]:
from langchain_community.document_loaders import PyPDFLoader

pdf_file = "/kaggle/input/fashion-data/fashion_data.pdf"

loader = PyPDFLoader(pdf_file)

In [3]:
# aquí estamos cargando un listado de documentos del pdf original (por página)
# debemos pasar una lista de texto al splitter
document = loader.load()

# páginas PDF cargadas
#print(f"El número de documentos cargados es: {len(document)}")

#document[0]
#type(document)

## **Text Splitting**

Dependencias

[How to recursively split text by characters](https://python.langchain.com/docs/how_to/recursive_text_splitter/)

[Documents Splitting with LangChain](https://www.kaggle.com/code/youssef19/documents-splitting-with-langchain)

In [4]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator= "\n",
    chunk_size= 1000,
    chunk_overlap= 150,
    length_function= len
)

docs = text_splitter.split_documents(document)

In [5]:
# comparamos el número de chunks creados con el de páginas del PDF inicial
print(len(docs), len(document))

37 21


In [6]:
# detalle chuncks creados
docs[3].page_content

'"Color": "Royal Blue",  \n        "Sizes Available": ["S", "M", "L"],  \n        "Features": "Floor -length, halter neck, with draped silhouette and side slit."  \n    }, \n    { \n        "Item": "Mountaineer Thermal Vest",  \n        "Type": "Jacket",  \n        "Style": "Sporty",  \n        "Season": "Winter",  \n        "Material": "Insulated synthetic",  \n        "Color": "Mountain Gray",  \n        "Sizes Available": ["M", "L", "XL"],  \n        "Features": "Lightweight, water -resistant, with thermal pockets and zip -front closure."  \n    }, \n    { \n        "Item": "Heritage Wool Sweater",  \n        "Type": "Shirt",  \n        "Style": "Casual",  \n        "Season": "Winter",  \n        "Material": "Wool",  \n        "Color": "Burgundy",  \n        "Sizes Available": ["S", "M", "L"],  \n        "Features": "Cable knit, crew neck, with ribbed trim and elbow patches."  \n    }, \n    { \n        "Item": "Nomad Leather Backpack",  \n        "Type": "Accessory",  \n        "St

### **Text Embedding**

[Hugging Face Models](https://python.langchain.com/docs/integrations/platforms/huggingface/)

In [7]:
%%capture
!pip install --upgrade --quiet  langchain sentence_transformers langchain_huggingface

In [8]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# Convertir chuncks a embeddings
try:
    embed = embeddings.embed_documents([doc.page_content for doc in docs])
    print("Vector de embeddings creado con éxito")
    
except Exception as e:
    print(f"Error creando Vector de embeddings: {e}")

Vector de embeddings creado con éxito


### **Store in Vector Database (Chroma)**